In [1]:
import cv2
import face_recognition
import pickle
import time
from tkinter import *
from PIL import Image
from PIL import ImageTk
from tkinter import filedialog

In [2]:
image_file = 'image/marathon_01.jpg'
encoding_file = 'encodings.pickle'
unknown_name = 'Unknown'
title_name = 'Face Recognition'
model_method = 'cnn'

In [3]:
def selectFile():
    file_name = filedialog.askopenfilename(initialdir="./image",title="Select file",\
                        filetypes = (("jpeg files", "*.jpg"),("all files", "*.*")))
    print('File name : ' , file_name)
    read_image = cv2.imread(file_name)
    image = cv2.cvtColor(read_image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    imgtk = ImageTk.PhotoImage(image=image)
    (height, width) = read_image.shape[:2]
    detectAndDisplay(read_image)

In [4]:
def detectAndDisplay(image):
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = face_recognition.face_locations(rgb, model=model_method)
    encodings = face_recognition.face_encodings(rgb, boxes)
    names = []
    for encoding in encodings:
        matches = face_recognition.compare_faces(data["encodings"], encoding)
        name = unknown_name
        if True in matches:
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}  
            for i in matchedIdxs:  
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            name = max(counts, key=counts.get) 
        names.append(name)
    for ((top, right, bottom, left), name) in zip(boxes, names):
        y = top - 15 if top - 15 > 15 else top + 15
        color = (0, 255, 0)
        if(name == unknown_name):
            color = (0, 0, 255)
            name = ''
        cv2.rectangle(image, (left, top), (right, bottom), color, 2)
        y = top - 15 if top - 15 > 15 else top + 15
        cv2.putText(image,name,(left, y),cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    imgtk = ImageTk.PhotoImage(image=image)
    detection.config(image=imgtk)
    detection.image = imgtk

In [5]:
main = Tk()
main.title(title_name)
main.geometry()
data = pickle.loads(open(encoding_file, "rb").read())
# 이미지 로딩
read_image = cv2.imread(image_file )
# RGB로 변환
image = cv2.cvtColor(read_image, cv2.COLOR_BGR2RGB)
# fromarray 사용
image = Image.fromarray(image)
# PhotoImage 사용
imgtk = ImageTk.PhotoImage(image=image)
# 윈도우에 제목라벨, 
label=Label(main, text=title_name)
label.config(font=("Courier", 18))
label.grid(row=0,column=0,columnspan=4)
#파일선택 버튼 (sticky : 상하좌우 경계선 확장 맞춤)
btn=Button(main,text="File Select", height=2, command=lambda:selectFile())
btn.grid(row=1, column=0, columnspan=4, sticky=(W, E)) 
# 그림 들어갈 라벨  각 한줄씩 차지하여 생성
detection=Label(main, image=imgtk)
detection.grid(row=2,column=0,columnspan=4)
# detectAndDisplay 호출
detectAndDisplay(read_image)
main.mainloop()